# Team Project, Week 6: Data Cleaning & Transforming

## Introduction

### Purpose:
This milestone focuses on working with query results, *not* designing schemas or practicing subqueries again.
We will use SQL to create a single analysis-ready dataset from our PWHL Hockey Database, load it into pandas, and clean and transform it so it is ready for analysis.

### Technique:
Instead of copying results out of DB Browser, we will use Python to connect to our `pwhl_hockey.db`, run the query, and immediately process the output by exploring the database as DataFrames. Each DataFrame should represent a meaningful analysis result (query) — not a table.

- Tables live in the database. 
- DataFrames live in Python for analysis.

We will only pull data into a DataFrame when we need it.

### Additional Files:

Individual files for steps not performed in this summative notebook can be found the GitHub repository here: [pwhl-database](https://github.com/alyzukas/pwhl-database/tree/main)

## Part A: Create an Analysis-Ready Dataset Using SQL
### Step 1: Light Data Expansion
Change applied in our updated sql files logged in Github (see above).

### Step 2: Write SQL JOIN Queries
One of two queries logged in Github (see above) will be loaded below.

## Part B: Load SQL Results into pandas

### Step 3: Load One Query Result


#### Connecting to the database

Current database file is titled: `week6_pwhl.db`
- Includes updated schema
- Includes updated sample data

In [4]:
# Import libraries
import sqlite3
import pandas as pd

# Connect to `week6_pwhl` SQLite database 
db_path = "/Users/alyssa.zukas/cpsc5071/week6_pwhl.db"  # -- INSERT YOUR PATH HERE  
conn = sqlite3.connect(db_path)

In [5]:
# Confirm we can read from connection
pd.read_sql("SELECT * FROM team LIMIT 5;", conn)

,team_key,name,nickname,team_code,division,date_founded
0,1,Boston Fleet,Fleet,BOS,1,2023-08-29
1,2,Minnesota Frost,Frost,MIN,1,2023-08-29
2,3,Montréal Victoire,Victoire,MTL,1,2023-08-29
3,4,New York Sirens,Sirens,NY,1,2023-08-29
4,5,Ottawa Charge,Charge,OTT,1,2023-08-29


In [6]:
# Test Query -- View all tables
tables = pd.read_sql_query("""
    SELECT name
    FROM sqlite_master
    WHERE type='table'
    ORDER BY name;
""", conn)

tables

,name
0,assist
1,game
2,goal
3,location
4,penalties
5,period
6,player
7,season
8,shot
9,team


In [7]:
# Test Queries -- Show Table Structure 
pd.read_sql("PRAGMA table_info(player);", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,player_key,INT,0,None,1
1,1,hometown,VARCHAR(100),1,None,0
2,2,birthdate,DATE,1,None,0
3,3,name,VARCHAR(100),1,None,0
4,4,shoots,VARCHAR(10),0,None,0
5,5,catches,VARCHAR(10),0,None,0


In [8]:
pd.read_sql("PRAGMA table_info(team);", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,team_key,INT,0,None,1
1,1,name,VARCHAR(255),1,None,0
2,2,nickname,VARCHAR(255),1,None,0
3,3,team_code,VARCHAR(255),1,None,0
4,4,division,INT,1,None,0
5,5,date_founded,DATE,1,None,0


In [9]:
pd.read_sql("PRAGMA table_info(game);", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,game_key,INT,0,None,1
1,1,game_date,DATE,1,None,0
2,2,game_number,INT,1,None,0
3,3,location_key,INT,1,None,0
4,4,attendance,INT,0,None,0
5,5,start_time,DATETIME,0,None,0
6,6,end_time,DATETIME,0,None,0
7,7,duration,DATETIME,0,None,0
8,8,status,VARCHAR(50),1,None,0
9,9,season_key,INT,1,None,0


The above test queries helps us:
- Check data types
- Confirm foreign key columns exist

In [10]:
# Test Query -- Row Count
tables = ["player", "team", "game", "shot", "goal", "assist", "penalties"]

for t in tables:
    print(t)
    display(pd.read_sql(f"SELECT COUNT(*) AS row_count FROM {t};", conn))

player


,row_count
0,36


team


,row_count
0,8


game


...

,row_count
0,11


assist


,row_count
0,21


penalties


,row_count
0,13


In the above test query we are able to confirm:
- Data exists
- No table accidentally empty
- Joins will produce meaningful output 

#### Load SQL Query results into Pandas


**QUERY 1: Shot-level dataset (one row per shot)** <br>
This query pulls information about every shot (details about the shooter, the goalie, and the blockers and assists when applicable). <br>
We have detailed information about the shooter and the shot to perform analysis about the shooter
we can compile data about shot efficiency, and analyze some basic defensive data from the blocker and goalie

After choosing **one** of our JOIN queries to become our main dataset, we will either:

1. **Paste the query text** into Python and run `pd.read_sql()`, **OR**
2. **Read the query text from the .sql file** (as a normal text file) and run `pd.read_sql()`.

To allow this notebook to be easily passed on to all members, we will **paste the query text into python** so each person does not have to insert their exact path to the file, and can instead just run this:


In [11]:
# QUERT 1 - SHOT-LEVEL DATASET
# Paste the query text 
query_1 = """
SELECT t.name as shooting_team
,gm.game_date
,gm.duration
-- shooter details
,shooter.name as shooter_name
,tn.position
,shooter.shoots as shoots_handed
,shooter.hometown
,shooter.birthdate
,tn.jersey_number
-- shot details
,s.shot_key 
,s.shot_type
,s.shot_time
,s.shot_quality
-- goalie details
,goalie.name as goalie_name
,s.is_goal
--blocker and assists
,blocker.name AS blocker_name
,pa.name AS assist1_name
,pa2.name as assist2_name
--goal details
,gl.is_power_play as is_power_play_goal
,gl.is_short_handed as is_short_handed_goal
,gl.is_empty_net as is_empty_net_goal
,gl.is_penalty_shot as is_penalty_shot_goal
,gl.is_insurance_goal
,gl.is_game_winning_goal
,s.x_location
,s.y_location
FROM shot s
INNER JOIN game gm 
	on s.game_key=gm.game_key
INNER JOIN player shooter 
	on s.shooter_key=shooter.player_key
INNER JOIN player goalie
	on s.goalie_key=goalie.player_key
LEFT JOIN player blocker
	on s.blocker_key=blocker.player_key
INNER JOIN tenure tn
    ON shooter.player_key = tn.player_key
   AND gm.season_key = tn.season_key
INNER JOIN team t
    ON tn.team_key = t.team_key
LEFT JOIN goal gl 
	ON s.game_key=gl.game_key
    AND s.shot_key=gl.shot_key
LEFT JOIN assist a 
	ON gl.game_key=a.game_key
	AND gl.goal_key=a.goal_key
    AND a.assist_number = 1
LEFT JOIN assist a2
	ON gl.game_key=a2.game_key
	AND gl.goal_key=a2.goal_key
    AND a2.assist_number = 2
LEFT JOIN player pa
	ON a.player_key=pa.player_key
LEFT JOIN player pa2
    ON a2.player_key=pa2.player_key
ORDER BY s.shot_key;
"""

In [12]:
# Option 2: Read in the query from .sql file
# DO NOT DO THIS
# just for notes
"""
with open("/Users/alyssa.zukas/cpsc5071/pwhl_hockey/Week6_PartA_1_2/query1_shots.sql", "r") as f:
    query_text = f.read()

df = pd.read_sql(query_text, conn)
"""

'\nwith open("/Users/alyssa.zukas/cpsc5071/pwhl_hockey/Week6_PartA_1_2/query1_shots.sql", "r") as f:\n    query_text = f.read()\n\ndf = pd.read_sql(query_text, conn)\n'

##### **Why this query is the better “main dataset”**

- It’s row-level (one row = one shot), which is ideal for cleaning + transformation.
- It includes real missingness (LEFT JOIN blocker/assists/goal flags), which is perfect for our missing data strategies requirements.
- It has lots of columns where we can do the Week 6 tasks easily:
    - trim/standardize names/strings
    - fix data types (dates/times/bools)
    - create new columns (full_name, shot_distance, shot_outcome label, etc.)

##### **Load it into Pandas as the main dataset: `df`**

In [13]:
# Load the query_1 into Pandas
df = pd.read_sql(query_1, conn)

In [14]:
df.head()

,shooting_team,game_date,duration,shooter_name,position,shoots_handed,hometown,birthdate,jersey_number,shot_key,...,assist1_name,assist2_name,is_power_play_goal,is_short_handed_goal,is_empty_net_goal,is_penalty_shot_goal,is_insurance_goal,is_game_winning_goal,x_location,y_location
0,Toronto Sceptres,2026-01-20,2000-01-01 02:31:00,Blayre Turnbull,F,R,"Stellarton, NS",1993-07-15,19,4,...,Claire Dalton,Kali Flanagan,0.0,0.0,0.0,0.0,0.0,0.0,513,196
1,Seattle Torrent,2026-01-18,2000-01-01 02:45:00,Alex Carpenter,G,L,"North Reading, MA",1994-04-13,31,16,...,Daryl Watts,Savannah Harmon,0.0,0.0,0.0,0.0,0.0,0.0,175,143
2,Boston Fleet,2026-01-18,2000-01-01 02:45:00,Megan Keller,C,L,"Farmington, MI",1996-05-01,77,36,...,Abby Newhook,Alina Müller,1.0,0.0,0.0,0.0,0.0,0.0,423,203
3,Boston Fleet,2026-01-18,2000-01-01 02:45:00,Megan Keller,C,L,"Farmington, MI",1996-05-01,77,41,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,434,217
4,Boston Fleet,2026-01-18,2000-01-01 02:45:00,Haley Winn,F,R,"Rochester, NY",2003-07-14,88,43,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,389,173


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   shooting_team         15 non-null     object 
 1   game_date             15 non-null     object 
 2   duration              15 non-null     object 
 3   shooter_name          15 non-null     object 
 4   position              15 non-null     object 
 5   shoots_handed         15 non-null     object 
 6   hometown              15 non-null     object 
 7   birthdate             15 non-null     object 
 8   jersey_number         15 non-null     int64  
 9   shot_key              15 non-null     int64  
 10  shot_type             15 non-null     object 
 11  shot_time             15 non-null     object 
 12  shot_quality          15 non-null     object 
 13  goalie_name           15 non-null     object 
 14  is_goal               15 non-null     int64  
 15  blocker_name          4 n

In [16]:
df.describe(include="all")

,shooting_team,game_date,duration,shooter_name,position,shoots_handed,hometown,birthdate,jersey_number,shot_key,...,assist1_name,assist2_name,is_power_play_goal,is_short_handed_goal,is_empty_net_goal,is_penalty_shot_goal,is_insurance_goal,is_game_winning_goal,x_location,y_location
count,15,15,15,15,15,15,15,15,15.000000,15.000000,...,11,10,11.000000,11.0,11.0,11.0,11.000000,11.000000,15.000000,15.000000
unique,3,2,2,13,5,2,13,13,NaN,NaN,...,11,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,Seattle Torrent,2026-01-20,2000-01-01 02:31:00,Megan Keller,F,L,"Farmington, MI",1996-05-01,NaN,NaN,...,Claire Dalton,Kali Flanagan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,8,10,10,2,5,9,2,2,NaN,NaN,...,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.533333,77.133333,...,NaN,NaN,0.272727,0.0,0.0,0.0,0.090909,0.090909,260.200000,157.666667
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.405701,58.907757,...,NaN,NaN,0.467099,0.0,0.0,0.0,0.301511,0.301511,198.371297,37.982452
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000000,4.000000,...,NaN,NaN,0.000000,0.0,0.0,0.0,0.000000,0.000000,39.000000,67.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.500000,42.000000,...,NaN,NaN,0.000000,0.0,0.0,0.0,0.000000,0.000000,83.500000,145.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.000000,55.000000,...,NaN,NaN,0.000000,0.0,0.0,0.0,0.000000,0.000000,175.000000,161.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.000000,98.500000,...,NaN,NaN,0.500000,0.0,0.0,0.0,0.000000,0.000000,428.500000,175.500000


In [17]:
df.shape

(15, 26)

In [18]:
df.dtypes

shooting_team            object
game_date                object
duration                 object
shooter_name             object
position                 object
shoots_handed            object
hometown                 object
birthdate                object
jersey_number             int64
shot_key                  int64
shot_type                object
shot_time                object
shot_quality             object
goalie_name              object
is_goal                   int64
blocker_name             object
assist1_name             object
assist2_name             object
is_power_play_goal      float64
is_short_handed_goal    float64
is_empty_net_goal       float64
is_penalty_shot_goal    float64
is_insurance_goal       float64
is_game_winning_goal    float64
x_location                int64
y_location                int64
dtype: object

## Part C: Data Cleaning and Transformation


As the main focus on this milestone, we will perform the following steps to clean and transform the data:
- **Handle Missing Data:** Identify missing values using `df.isnull().sum()` and apply at least two missing-data strategies (make sure to explain).
- **Clean and Standardize Columns:** Perform *at least* three of the following:
    - Trim whitespace from strings
    - Standardize capitalization
    - Fix data types (dates, integers, floats)
    - Rename unclear column names
    - Remove duplicates if present
- **Transform the Data:** Create *at least two new columns* that improve usability or analysis, show a short preview so it is clear what changed, and save it in `cleaned_data.csv`.

### Step 4: Handle Missing Data 

In [19]:
# Sanity Check (before changes) 
# should only be missing blocker name if the goal was sucessful (11)
# should only be missing goal fields if the shot was not a goal (4)
# should only be missing assist1 if there were no assists or there was no goal (4)
# should only be missing assist2 if there was only one asssist on a goal (4 no goal, 1 goal with only one assist)

goal_flag_cols = [
    "is_power_play_goal",
    "is_short_handed_goal",
    "is_empty_net_goal",
    "is_penalty_shot_goal",
    "is_insurance_goal",
    "is_game_winning_goal"
]

conditional_name_cols = [
    "blocker_name", 
    "assist1_name", 
    "assist2_name"
]

all_null_cols = goal_flag_cols + conditional_name_cols

df[all_null_cols].isnull().sum()

is_power_play_goal       4
is_short_handed_goal     4
is_empty_net_goal        4
is_penalty_shot_goal     4
is_insurance_goal        4
is_game_winning_goal     4
blocker_name            11
assist1_name             4
assist2_name             5
dtype: int64

##### Identify Missing Values

In [20]:
# Idenftify Missing Values
df.isnull().sum().sort_values(ascending=False)

blocker_name            11
assist2_name             5
is_game_winning_goal     4
is_insurance_goal        4
is_penalty_shot_goal     4
is_empty_net_goal        4
is_short_handed_goal     4
is_power_play_goal       4
assist1_name             4
shooting_team            0
game_date                0
x_location               0
is_goal                  0
goalie_name              0
shot_quality             0
shot_time                0
shot_type                0
shot_key                 0
jersey_number            0
birthdate                0
hometown                 0
shoots_handed            0
position                 0
shooter_name             0
duration                 0
y_location               0
dtype: int64

In [21]:
# Cleaner view to show missing values 
missing = df.isnull().sum()
missing[missing > 0].sort_values(ascending=False)

blocker_name            11
assist2_name             5
assist1_name             4
is_power_play_goal       4
is_short_handed_goal     4
is_empty_net_goal        4
is_penalty_shot_goal     4
is_insurance_goal        4
is_game_winning_goal     4
dtype: int64

In [22]:
# Check categorical filled values
df[conditional_name_cols]

,blocker_name,assist1_name,assist2_name
0,None,Claire Dalton,Kali Flanagan
1,Jill Saulnier,Daryl Watts,Savannah Harmon
2,None,Abby Newhook,Alina Müller
3,Mariah Keopple,None,None
4,Lexie Adzija,None,None
5,None,Hannah Bilka,Alex Carpenter
6,Blayre Turnbull,None,None
7,None,None,None
8,None,Cayla Barnes,None
9,None,Brooke Bryant,Cayla Barnes


##### Apply Missing-Data Stategies

In [23]:
# Fill Categorical missing values with "UNKNOWN"
df[conditional_name_cols] = df[conditional_name_cols].fillna("None")

In [24]:
# Fill Goal Flags with 0 
df[goal_flag_cols] = df[goal_flag_cols].fillna(0).astype(int)

If the shot was not a goal, those fields should logically be 0.

A LEFT JOIN means:
- If the shot was a goal $\rightarrow$ join succeeds $\rightarrow$ flags come in as 0 or 1.
- If the shot was NOT a goal $\rightarrow$ no matching goal row $\rightarrow$ all goal columns become NULL.

So in this case, a NULL does NOT mean: <br>
“we don’t know”

It means: <br>
“this shot was not a goal, so there is no goal record.”

##### Explanation 

Missing values were identified using `df.isnull().sum()` after loading the dataset into pandas. The missing values were found in blocker and assist columns, as well as goal-related flags. These missing values are expected due to the relational design and use of LEFT JOINs.

For example, 11 shots did not have a blocker, 4 shots did not have a primary assist records, 1 shot did have a primary assistn but no secondary. These NULL values do not represent incomplete data. Rather, they reflect normal hockey events in which many shots are unblocked and not all goals involve assists. These categorical fields were filled with string `None` to indicate that the event did not occur while preserving the full shot-level dataset.

Four rows contained missing goal-related flags. These NULL values occurred because shots that were not goals do not have corresponding rows in the goal table. Since these columns represent binary conditions that apply only to goal events, missing values were filled with `0` to indicate that the condition was not met. This preserves logical consistency and ensures accurate aggregation during analysis.

Overall, the missing values in this dataset were structural rather than random, and the chosen strategies preserve both relational integrity and analytical usability.

##### Sanity Check (after changes)

In [25]:
#check that NA no longer appears for those columns we filled
missing = df.isnull().sum()
missing[missing > 0].sort_values(ascending=False)

Series([], dtype: int64)

### Step 5: Clean and Standardize Columns
#### Rename Columns for Clarity
This was actually performed within the SQL query itself, where informative aliases were assigned to columns in the select statement using syntax keyword "as". One example include renaming "shoots" to "shoots_handed".

#### Correct Data Types
Convert object columns to string when appropriate, object columsn to datetime when appropriate, and float to integer when appropriate.


In [26]:
to_string_cols = [
    "shooting_team",
    "shooter_name",  
    "shoots_handed",
    "position",
    "hometown",
    "shot_type",
    "shot_quality",
    "goalie_name",
    "blocker_name",
    "assist1_name",
    "assist2_name"
]

to_datetime_cols = [
    "duration",
    "game_date",
    "birthdate",
    "shot_time"
]

to_int_cols = [
    "is_power_play_goal",
    "is_short_handed_goal",
    "is_empty_net_goal",
    "is_penalty_shot_goal",
    "is_insurance_goal",
    "is_game_winning_goal"
]

for i in to_string_cols:
    df[i] = df[i].astype("string")

for i in to_datetime_cols:
    df[i] = pd.to_datetime(df[i])

for i in to_int_cols:
    df[i] = df[i].astype("int64")

#confirm appropriate datatype applied
df.dtypes

shooting_team           string[python]
game_date               datetime64[ns]
duration                datetime64[ns]
shooter_name            string[python]
position                string[python]
shoots_handed           string[python]
hometown                string[python]
birthdate               datetime64[ns]
jersey_number                    int64
shot_key                         int64
shot_type               string[python]
shot_time               datetime64[ns]
shot_quality            string[python]
goalie_name             string[python]
is_goal                          int64
blocker_name            string[python]
assist1_name            string[python]
assist2_name            string[python]
is_power_play_goal               int64
is_short_handed_goal             int64
is_empty_net_goal                int64
is_penalty_shot_goal             int64
is_insurance_goal                int64
is_game_winning_goal             int64
x_location                       int64
y_location               

#### Trim Whitespace
For each string datatype, ensure unnecessary white space has been cleared.

In [27]:
for i in to_string_cols:
    df[i] = df[i].str.strip()

### Step 6: Transform the Data


#### Split Hometown into Town and State / Territory

In [28]:
#before split this column holds town comma state/territory
df.hometown

0        Stellarton, NS
1     North Reading, MA
2        Farmington, MI
3        Farmington, MI
4         Rochester, NY
5        Sun Valley, ID
6          Brampton, ON
7            Alpena, MI
8           Coppell, TX
9            London, ON
10           London, ON
11       Burlington, MA
12           Milton, ON
13      Scarborough, ON
14           Barrie, ON
Name: hometown, dtype: string

In [29]:
#split
df[['hometown','home_state_or_territory']] = df.hometown.str.split(",", n=1, expand=True)

In [30]:
#show impact 
df[['hometown','home_state_or_territory']]

,hometown,home_state_or_territory
0,Stellarton,NS
1,North Reading,MA
2,Farmington,MI
3,Farmington,MI
4,Rochester,NY
5,Sun Valley,ID
6,Brampton,ON
7,Alpena,MI
8,Coppell,TX
9,London,ON


#### Calulate Game Duration in only Hours 

In [31]:
#calulate duration of game in only hours from the datetime hour minute column
#show before
df['duration']

0    2000-01-01 02:31:00
1    2000-01-01 02:45:00
2    2000-01-01 02:45:00
3    2000-01-01 02:45:00
4    2000-01-01 02:45:00
5    2000-01-01 02:31:00
6    2000-01-01 02:31:00
7    2000-01-01 02:31:00
8    2000-01-01 02:31:00
9    2000-01-01 02:45:00
10   2000-01-01 02:31:00
11   2000-01-01 02:31:00
12   2000-01-01 02:31:00
13   2000-01-01 02:31:00
14   2000-01-01 02:31:00
Name: duration, dtype: datetime64[ns]

In [32]:
#calculate
df['duration_hours'] = df['duration'].dt.hour + df['duration'].dt.minute / 60 

In [33]:
#show after
df['duration_hours'] 

0     2.516667
1     2.750000
2     2.750000
3     2.750000
4     2.750000
5     2.516667
6     2.516667
7     2.516667
8     2.516667
9     2.750000
10    2.516667
11    2.516667
12    2.516667
13    2.516667
14    2.516667
Name: duration_hours, dtype: float64

## Export the Clean Data 

In [35]:
# Export cleaned dataset
df.to_csv("week6_cleaned_data.csv", index=False)

print("Cleaned dataset saved as week6_cleaned_data.csv")

Cleaned dataset saved as week6_cleaned_data.csv


## Part D: Reflection (Short Write-Up)
`readme.txt` can be found at referenced location.

## Close Connection
Finished will all database work in this assignment, it is best practice to close our connection.

In [262]:
conn.close()